# Домашнее задание 4. Конструирование текстовых признаков из твитов пользователей

## Сбор данных

Первый этап - сбор твитов пользователей. Необходимо подключаться к Twitter API и запрашивать твиты по id пользователя. 
Подключение к API подробно описано в ДЗ 1.

In [34]:
import twitter
import matplotlib.pyplot as plt
import random
import re

CONSUMER_KEY = "JU3TRo8VDXB6521Y1z9dCkjtG"
CONSUMER_SECRET = "Q9tFFbdEecuNXmKdovICMP0sZ5r4a6iglzzB8bywyIQtQXF7LK"

ACCESS_TOKEN_KEY = "780472030546292736-B9OjRHVbOdPBomYYWpxa54vqHnFpvLZ"
ACCESS_TOKEN_SECRET = "6VM3hyw2QM8iG1z9DY1uoPrcG76KkU5c9RaSKKT4v8me6"

api = twitter.Api(consumer_key=CONSUMER_KEY, 
                  consumer_secret=CONSUMER_SECRET, 
                  access_token_key=ACCESS_TOKEN_KEY, 
                  access_token_secret=ACCESS_TOKEN_SECRET)

Для получения твитов пользователя может быть использован метод GetUserTimeline из библиотеки python-twitter. Он позволяет получить не более 200 твитов пользователя. По каждому пользователю достаточно собрать 200 твитов.

Метод имеет ограничение по количеству запросов в секунду. Для получения информации о промежутке времени, которое необходимо подождать для повторного обращения к API может быть использован метод `GetSleepTime`. Для получения информации об ограничениях запросов с помощью метода `GetUserTimeLine` необходимо вызывать `GetSleepTime` с параметром "statuses/user_timeline".

Метод GetUserTimeline возвращает объекты типа Status. У этих объектов есть метод AsDict, который позволяет представить твит в виде словаря.

Id пользователей необходимо считать из файла, как было сделано в ДЗ 1.

Необходимо реализовать функцию `get_user_tweets(user_id)`. Входной параметр - id пользователя из файла. Возвращаемое значение - массив твитов пользователя, где каждый твит представлен в виде словаря. Предполагается, что информация о пользователе содержится в твитах, которые пользователь написал сам. Это означает, что можно попробовать отфильтровать ответы другим пользователям, ссылки и ретвиты, а так же картинки и видео, так как наша цель - найти текстовую информацию.

In [35]:
def get_user_tweets(user_id):
    reslist = []
    try:
        statuses = api.GetUserTimeline(user_id = user_id, count=200, trim_user=True, 
                                       exclude_replies=True, include_rts=False)
        for status in statuses:
            d = status.AsDict()
            if not ('retweeted_status' in d) and (d['lang']=='en'):
                res = {'lang': d['lang'], 
                 'favorited': d['favorite_count']>0 if 'favorite_count' in d else False, 
                 'truncated': d["truncated"] if "truncated" in d else False, 
                 'text': d['text'], 
                 'created_at': d['created_at'], 
                 'retweeted': d['retweet_count']>0 if 'retweet_count' in d else False, 
                 'source': d['source'] if 'source' in d else None, 
                 'user': d['user'], 
                 'id': d['id']}
                reslist.append(res)
    except twitter.TwitterError as err:
        if err.message=="Rate limit exceeded":
            raise
        if err.message!="Not authorized.":
            print "\nError: ",err
        return reslist
    return reslist

## Разбор текста твита

Обработка текста предполагает разбиение текста на отдельные элементы - параграфы, предложения, слова. Мы будем преобразовывать текст твита к словам. Для этого текст необходимо разбить на слова. Сделать это можно, например, с помощью регулярного выражения.

Необходимо реализовать функцию, `get_words(text)`. Входной параметр - строка с текстом. Возвращаемое значение - массив строк (слов). Обратите внимание, что нужно учесть возможное наличие пунктуации и выделить по возможности только слова. 

In [36]:

def get_words(text):
    text = text.lower()
    find = re.findall(r'[^\\=\-?~$%&/\[\]+><{}_^(@)*#\s!,.?":;0-9]+', text)
    return find

In [37]:
print get_words("Here are different words!")

['here', 'are', 'different', 'words']


Далее полученные слова необходимо привести к нормальной форме. То есть привести их к форме единственного числа настоящего времени и пр. Сделать это можно с помощью библиотеки nltk. Информацию по загрузке, установке библиотеки и примерах использования можно найти на сайте http://www.nltk.org/

Для загрузки всех необходимых словарей можно воспользоваться методом download из библиотеки nltk.

In [38]:
import nltk

nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

wnl = WordNetLemmatizer()
list_stopwords = stopwords.words()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Olenka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Olenka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Для дальнейшей обработки слова должны быть приведены к нижнему регистру. 

Для приведения к нормальной форме можно использовать `WordNetLemmatizer` из библиотеки nltk. У этого класса есть метод `lemmatize`.

Также необходимо убрать из текста так называемые стоп-слова. Это часто используемые слова, не несущие смысловой нагрузки для наших задач. Сделать это можно с помощью `stopwords` из nltk.corpus

Необходимо реализовать функцию `get_tokens(words)`. Входной параметр - массив слов. Возвращаемое значение - массив токенов.

In [39]:
def get_tokens(words):
    list_tokens = [wnl.lemmatize(word) for word in words if wnl.lemmatize(word) not in list_stopwords]
    return list_tokens

Необходимо реализовать функцию `get_tweet_tokens(tweet)`. Входной параметр - текст твита. Возвращаемое значение -- токены твита. 

In [40]:
def get_tweet_tokens(tweet):
    return get_tokens(get_words(tweet))

Необходимо реализовать функцию `collect_users_tokens()`. Функция должна сконструировать матрицу признаков пользователей. В этой матрице строка - пользователь. Столбец - токен. На пересечении - сколько раз токен встречается у пользователя.
Для построения матрицы можно использовать `DictVectorizer` из `sklearn.feature_extraction`.

In [41]:
def collect_users_tokens(df_users):
    """returns users list and list of user dicts. Each dict contains frequence of user tokens"""
    list_users = []
    list_dict_tokens = []
    # Noise found on tag-cloud
    list_noise = ['http', 'don\'t', 'i\'m', 'it', 'it\'s', 'amp']
    
    for user in df_users.uid:
        user_tweets = get_user_tweets(user)
        dict_tokens = {}
        for tweet in user_tweets:
            list_tokens = get_tweet_tokens(tweet["text"])
            for token in list_tokens:
                if token in dict_tokens:
                    dict_tokens[token] =  dict_tokens[token] + 1
                else:
                    if len(token) > 1 and token not in list_noise:
                        dict_tokens[token] = 1
        list_users.append(user)
        list_dict_tokens.append(dict_tokens)
        
    return list_users, list_dict_tokens
    

In [42]:
import pandas as pd
from collections import defaultdict
import json
from sklearn.feature_extraction import DictVectorizer


TRAINING_SET_URL = "train.csv"
EXAMPLE_SET_URL = "test.csv"
df_users_train = pd.read_csv(TRAINING_SET_URL, sep=",", header=0, names=["uid", "cls"]
df_users_ex = pd.read_csv(EXAMPLE_SET_URL, sep=",", header=0, names=["uid"])
df_users_ex['cls'] = None
df_users = pd.concat([df_users_train, df_users_ex])

users, users_tokens = collect_users_tokens(df_users)
v = DictVectorizer()
vs = v.fit_transform(users_tokens)


Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that page does not exist.', u'code': 34}]

Error:  [{u'message': u'Sorry, that p

ConnectionError: ('Connection aborted.', BadStatusLine("''",))

In [57]:
import numpy as np
np.savez("files/out_4.dat", data=vs, users=users, users_tokens=users_tokens )

Далее для получения представления о полученной информацию о токенах предлагается отобразить ее в виде облака тэгов. [Подсказка](http://anokhin.github.io/img/tag_cloud.png). 

In [58]:

from wordcloud import WordCloud

%matplotlib inline

def draw_tag_cloud(v, vs, users_tokens):
    """Draws tag cloud of found tokens"""
    freqs = sum((Counter(dict(x)) for x in users_tokens), Counter()).items()
    wc = WordCloud(max_words=1000, margin=10, background_color='white',
        random_state=1).generate_from_frequencies(freqs)
    default_colors = wc.to_array()
    plt.figure()
    plt.imshow(default_colors)
    plt.axis("off")
    plt.show()
    return

draw_tag_cloud(v, vs, users_tokens)